In [1]:
import os
print(os.getcwd())
# change working directory to project root
import platform
if platform.system() is "Windows":
    os.chdir('G:\\sven\\git\\cardio')
else:
    os.chdir('/Users/minority/Code/Git/cardio')
print(os.getcwd())
from src.utils.utils_io import Console_and_file_logger, ensure_dir
from src.utils.file_wrapper.app.Files import Basefile, Dicomfile
from src.utils.file_wrapper.app.IO import FileWalker
import logging
Console_and_file_logger('extract_segmentations_2', logging.INFO)

import re
#import nrrd
import numpy as np
from scipy import stats
from collections import OrderedDict
import json
import SimpleITK as sitk
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from collections import Counter
import cv2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# specific imports
import xml.etree.ElementTree as ET


G:\sven\git\cardio\notebooks
G:\sven\git\cardio


2019-01-18 14:14:49,022 INFO -------------------- Start --------------------
2019-01-18 14:14:49,023 INFO Filename: extract_segmentations_2
2019-01-18 14:14:49,024 INFO Log directory: ./logs/


Working directory: G:\sven\git\cardio.
Log dir: ./logs/


In [2]:
class Contour():
    
    def __init__(self, uid, image_size, pixel_size, points, sub_pixel_res, tag):
        self.uid = uid
        self.image_size = image_size
        self.pixel_size = pixel_size
        self.points = points
        self.sub_pixel_res = sub_pixel_res
        self.tag = tag
        
    uid = 0  
    image_size = (0,0)
    pixel_size = (0,0)
    sub_pixel_res = 0
    points = []

In [3]:
def contour_factory(contours, tag):
    """
    Expects a list of segmentation-elements
    Reads/collects all neccessary data for each contour-element
    Wraps each contour information in a contour-object,
    builds a list of Contours and returns them
    Params: list of contour-xml tags:
    e.g. call:
    xml_file = 'D:\\git\\cardio\\data\\processed\\Segmentations\\0AE4R74L190001011238291.3.12.2.1107.5.99.2.1013.30000008030612131778100072179.cvi42wsx'
    with open(xml_file) as fp:
    soup = BeautifulSoup(fp, 'lxml')
    saendocardialContour = soup.find_all('hash:item', {'hash:key' : 'saendocardialContour'})
    
    """
    cont = {}
    logging.info('building {} contours'.format(len(contours)))
    for contour in contours:
        #print(contour)
        uid = contour.parent.parent.attrs.get('hash:key')
        image_width = contour.find('hash:item', {'hash:key' : 'ImageSize'}).find('size:width').text
        image_height = contour.find('hash:item', {'hash:key' : 'ImageSize'}).find('size:height').text
        pixel_width = contour.find('hash:item', {'hash:key' : 'PixelSize'}).find('size:width').text
        pixel_height = contour.find('hash:item', {'hash:key' : 'PixelSize'}).find('size:height').text
        sub_pixel = contour.find('hash:item', {'hash:key' : 'SubpixelResolution'}).text
        points_t = contour.find('hash:item', {'hash:key' : 'Points'})
        points_x = [point.text for point in points_t.find_all('point:x')]
        points_x[:] = [int(int(x) / int(sub_pixel)) for x in points_x]
        points_y = [point.text for point in points_t.find_all('point:y')]
        points_y[:] = [int(int(y) / int(sub_pixel)) for y in points_y]
        points = list(zip(points_x, points_y))
        
        
        # create a Contour class for each contour
        cont[str(uid)] = Contour(uid, 
                                     (image_width, image_height), 
                                     (pixel_width, pixel_height),
                                    points,
                                    sub_pixel, tag)
    return cont

In [4]:
def extract_contours(soup):
    """
    Expects the Segmentation-XML-File as Beautifulsup-object
    Returns: a dictionary with segmentation-tag : list of segmentation-tag elements
    """
    c = {}
    contours = {'saendocardialContour': [],
           'saepicardialContour' : [],
           'sarvendocardialContour': [],
           'sarvepicardialContour': []}
    
    # Find root elements of each contour tag
    for tag, contour_elements in contours.items():
        logging.info('read: {}'.format(tag))
        contours[tag] = soup.find_all('hash:item', {'hash:key' : tag})
        logging.info('found: {} contours'.format(len(contours[tag])))
        
    
    """
    convert to objects
    Expects a dictionary object with:
    segmentation-tag : list of segmentation-tag elements
    converts the list of segmentation-elements
    into a flat dictionary with {uid : [contour]}
    """

    for tag, contour_elements in contours.items():
        if len(contour_elements) > 0:
            logging.info('extract all {} segmentations'.format(tag))
            # transform all contour elements in contour objects
            contours = contour_factory(contour_elements, tag)
            # update contours dictionary
            # we might have multiple contours per slice
            for uid, cont in contours.items():      
                if uid in c.keys():
                    c[uid].append(cont)
                else:
                    c[uid] = [cont]

    return c


In [5]:
class GetSegmenationDicoms(Dicomfile):
    """
    Used to filter all dicom images which are part of the segmentation 
    by comparing the series id with the valid series ids injected by the filewalker
    save them to dest
    returns sitk-image
    """

    def action(self):
        
        if self.is_part_of_volume():
            # save to destination path
            self.save(self.destination)
            return self
    
    def is_part_of_volume(self):
        """
        check if series-id is in valid series ids
        valid series ids are capturedinjected by filewalker to the dicomwrapper class
        and available via self.args
        """
        series_id = self.get_metadata_maybe('0020|000e')
        if series_id in self.args:
            return True
        else:
            return False
    
    def merge(self, a, b):
        if isinstance(a, dict):
            a = []
            a.append(b)
        else:
            a.append(b)
        return a
        
    
        

In [6]:
class GetDicomSeriesIDs(Dicomfile):
    """
    check if dicom file is part of the segmentation
    all valid segmentations are injected by the filewalker
    returns the series id of this sop
    """
    
    def action(self):
        if self.is_segment():
            #return the series-id
            series_id = self.get_metadata_maybe('0020|000e')
            return series_id
    
    def is_segment(self):
        # check if dicom-series is in contour-dict (injected by filewalker)
        uid = self.get_metadata_maybe('0008|0018')
        if uid in self.args.keys():
            return True
        else:
            return False
    
    def merge(self, a, b):
        if isinstance(a, dict):
            a = []
            a.append(b)
        else:
            a.append(b)
        return a

In [7]:
def sort_dicoms(dicom_images, sort_for_time=True):
    
    """
    Sort all slices first after time, then origin
    """
    
    logging.debug('images to sort: {}'.format(len(dicom_images)))  

    def origin_sort(dicom_wrapper):
        origin = dicom_wrapper.sitk_img.GetOrigin()
        time = dicom_wrapper.get_metadata_maybe('0018|1060')
        #logging.info(origin)
        if sort_for_time:
            return (float(time), origin[2], origin[1], origin[0])
        else:
            return (origin[2], origin[1], origin[0])
    
    sorted_dicom_images = sorted(dicom_images, key=origin_sort)
    
    return sorted_dicom_images
    

In [8]:
def get_timesteps(dicom_images):
    """
    calculate the timesteps of one volume by summing up the origins
    all slices with same origin represents the timesteps of one slice
    """
    
    origins = [img.sitk_img.GetOrigin() for idx, img in enumerate(dicom_images)]
    counter = Counter(origins)
    steps = counter[next(iter(counter))]
    return steps

In [9]:
def get_contour_old(sitk_img, contour_dict, contour_only=False):
    
    """
    tries to find a contour object for this dicom slice
    writes contour into dicom slice if one is given and contour_only = False
    Else return only contour as slice
    returns (modified) dicom slice
    """

    
    tags = {'saendocardialContour': 1,
           'saepicardialContour' : 2,
           'sarvendocardialContour': 3,
           'sarvepicardialContour': 0}
    
    # check if this slice SOP-UID is in the contour dictionary
    contour = contour_dict.get(sitk_img.GetMetaData('0008|0018'), False)
    if contour: # contour for slice given
        #print('contour found')
        if contour_only: # returns only the contour,create a slice with zeros
            temp_nda = sitk.GetArrayFromImage(sitk_img)
            shape = temp_nda.shape
            dtype = temp_nda.dtype
            nda = np.zeros(shape, dtype=dtype)
        else: # draws the contour into the slice
            nda = sitk.GetArrayFromImage(sitk_img)
            
        # write contour points with given values
        for cont in contour:
            for point in cont.points:
                nda[0, point[1], point[0]] = tags[cont.tag]
                
            contour_img = sitk.GetImageFromArray(nda)

            # copy image information
            for tag in sitk_img.GetMetaDataKeys():
                try:
                    value = sitk_img.GetMetaData(tag)
                    contour_img.SetMetadata(tag, value)
                except:
                    pass
        return contour_img
    
    else: # no contour given, but only contour ordered --> return empty slice
        if contour_only:
            temp_nda = sitk.GetArrayFromImage(sitk_img)
            nda = np.zeros(temp_nda.shape, dtype=temp_nda.dtype)
            contour_img = sitk.GetImageFromArray(nda)

            # copy image information
            for tag in sitk_img.GetMetaDataKeys():
                try:
                    value = sitk_img.GetMetaData(tag)
                    contour_img.SetMetadata(tag, value)
                except:
                    pass
        
            return contour_img
        else: # no contour found but mixed modus --> return slice
            return sitk_img

In [10]:
def get_contour(sitk_img, contour_dict, mixed_mode=False, mask=False):
    
    """
    tries to find a contour object for this dicom slice
    writes contours/masks into dicom slice
    
    sitk_img = sitk-image
    contour_dict = a flat dictionary with {uid : [contours]}
    class Contour():
        self.uid = uid
        self.image_size = image_size
        self.pixel_size = pixel_size
        self.points = points
        self.sub_pixel_res = sub_pixel_res
        self.tag = tag
    

    mixed_mode = True --> Use image slice as background
    mixed_mode = False --> Use emtpy slice as background
    
    mask = True --> fill contour, create masks
    mask = False --> Use plain contours
    
    returns (modified) dicom slice
    """

    # values to use for contour drawing
    tags = {'saendocardialContour': 1,
           'saepicardialContour' : 2,
           'sarvendocardialContour': 3,
           'sarvepicardialContour': 4}
    
    # check if this slice SOP-UID is in the contour dictionary
    # check if contour is given for this slice
    contour = contour_dict.get(sitk_img.GetMetaData('0008|0018'), False)
    
    if contour: # contour for slice given contour = list of contour objects
        
        # go through contours in reverse order, to draw first the outer line, than inner
        contour = reversed(contour) # rvepi, rvendo, epi, endo
        
        if mixed_mode: # use slice as background   
            nda = sitk.GetArrayFromImage(sitk_img)

        else: # use empty slice as background
            temp_nda = sitk.GetArrayFromImage(sitk_img)
            shape = temp_nda.shape
            dtype = temp_nda.dtype
            nda = np.zeros(shape, dtype=dtype)
        
        if mask: # fill contours, create masks
            nda_ = nda[0,:,:]
            for cont in contour:
                cv2.fillPoly(nda_, pts =[np.array(cont.points)], color=tags[cont.tag])
                nda = nda_[np.newaxis, :, :]
        
        else: # write contour points with given tag-values
            
            for cont in contour:
                for point in cont.points:
                    nda[0, point[1], point[0]] = tags[cont.tag]
            
                
        # copy image information
        contour_img = sitk.GetImageFromArray(nda)
        for tag in sitk_img.GetMetaDataKeys():
            try:
                value = sitk_img.GetMetaData(tag)
                contour_img.SetMetadata(tag, value)
            except:
                pass
        return contour_img
    
    else: # no contour given
        
        if mixed_mode: # mixed mode = True --> return image slice
            return sitk_img
            
        else: # mixed_mode = False --> return empty slice
            
            temp_nda = sitk.GetArrayFromImage(sitk_img)
            nda = np.zeros(temp_nda.shape, dtype=temp_nda.dtype)
            contour_img = sitk.GetImageFromArray(nda)

            # copy image information
            for tag in sitk_img.GetMetaDataKeys():
                try:
                    value = sitk_img.GetMetaData(tag)
                    contour_img.SetMetadata(tag, value)
                except:
                    pass
        
            return contour_img
            

In [11]:
def get_valid_dicom_image(contours, args):
    
    # get a list of all series ids
    logging.info('----------------    Search {} series-ids for dicom directory ----------------')
    series_ids = FileWalker(GetDicomSeriesIDs, src_dir=args['src'], dest_dir=args['dest'], args=contours).open_files
    logging.info('----------------    Found {} series-ids in dicom directory ----------------'.format(len(set(series_ids))))
      
    # load and filter all dicom images with series-id in seried_ids as dicom_wrapper-classes
    logging.info('----------------    Load all dicom images with given series-ids {} -----------------')
    dicom_images = FileWalker(GetSegmenationDicoms, src_dir=args['src'], dest_dir=args['dest'], args=series_ids).open_files
    logging.info('----------------    All dicoms loaded: {} -----------------'.format(len(dicom_images)))
    return dicom_images

In [12]:
def mask_slice(img):
    
    """
    gets a 2d image slice with contours
    First version with start end end index
    doesnt work well with open contours
    0 = background
    1 = saendocardialContour
    2 = saepicardialContour
    3 = sarvendocardialContour
    
    Returns: masked 2d slice
    RV = 3
    LV = 2
    Background = 0
    """
    
    cls = 3
    mask_as = 3
    
    cls_lv_endo = 1
    mask_as_lv_endo = 0
    
    start_cls_lv_epi=3 
    end_cls_lv_epi=2 
    mask_as_lv_epi=2
    # make a local copy to avoid side effects
    img_copy = img.copy()
    logging.debug(img.shape)
    
    # take the middle area
    for i in range(img.shape[0]):
        start_idx_rv = img.shape[1]
        end_idx_rv = 0
        start_idx_lv_endo = img.shape[1]
        end_idx_lv_endo = 0
        start_idx_lv_epi = 0
        end_idx_lv_epi = 0
        # for each row
        for j in range(img.shape[1]):

            # get start index for rv
            if img[i,j] == cls and j < start_idx_rv:
                start_idx_rv = j
            # get end index for rv
            elif img[i,j] == cls and j >= end_idx_rv:
                end_idx_rv = j
                    
            # get start index of left endo
            if img[i,j] == cls_lv_endo and j < start_idx_lv_endo:
                start_idx_lv_endo = j
            # get end index of left endo
            elif img[i,j] == cls_lv_endo and j >= end_idx_lv_endo:
                end_idx_lv_endo = j
                
            # get start index - find right end of rv contour as start for left epi
            if img[i,j] == start_cls_lv_epi and j > start_idx_lv_epi:
                start_idx_lv_epi = j
            # get end index of left epi
            elif img[i,j] == end_cls_lv_epi and j > end_idx_lv_epi:
                end_idx_lv_epi = j
        
        img_copy[i,start_idx_rv:end_idx_rv] = mask_as
        img_copy[i,start_idx_lv_epi:end_idx_lv_epi] = mask_as_lv_epi
        img_copy[i,start_idx_lv_endo:end_idx_lv_endo] = mask_as_lv_endo
    

    return img_copy

In [13]:
def get_contour_points(contour_slice, value):
    """
    Extract the contour points from a 2d numpy array
    contour_slice: 2D numpy array with contour points on it
    value: value of the contour weare searching for
    returns: numpy array with points as tuples
    """
    #print(contour_slice.shape)
    points = []
    for i in range(contour_slice.shape[0]):
        for j in range(contour_slice.shape[1]):
            if contour_slice[i,j] == value:
                points.append((j,i))        
    return np.array(points)

In [14]:
def fill_contour(contour_slice):
    """
    gets a 2d numpy image-slice with contours
    returns mased 2d numpy image slice
    currently used
    0 = background
    1 = saendocardialContour
    2 = saepicardialContour
    3 = sarvendocardialContour
    
    Returns: masked 2d slice
    RV = 3
    LV = 2
    Background = 0
    """
    
    values = [3,2]
    c = contour_slice
    slice = c.copy()
    
    # use the rv contour to get a boarder for the epicardial contour
    points_rv = get_contour_points(c, 3)
    if len(points_rv > 0):
        cv2.fillPoly(slice, pts =[points_rv], color=2)
        
    # draw rv and epi as one big epi mask
    points_ep = get_contour_points(slice, 2)
    if len(points_ep > 0):
        cv2.fillPoly(slice, pts =[points_ep], color=2)
        
    # delete the inner endocardial part
    points_en = get_contour_points(c, 1)
    if len(points_en > 0):
        cv2.fillPoly(slice, pts =[points_en], color=0)
        
    # overwrite the rv part
    if len(points_rv > 0):
        cv2.fillPoly(slice, pts =[points_rv], color=3)
    
    return slice

In [15]:
def build_volume(contour_as_dict, dicoms_sorted, args):
    
    """
    Builds 4 x 4D nrrd volumes:
    - dicom images sorted
    - contours sorted
    - dicom and contours mixed
    - masks sorted
    """
    
    image_slices = []
    image_slices_clean = []
    contour_slices = []
    mask_slices = []

    
    timesteps = int(get_timesteps(dicoms_sorted))
    slices = int(len(dicoms_sorted)/timesteps)
    logging.info('Building volumes for patient: {}'.format(args['patient']))
    logging.info('images: {}'.format(len(dicoms_sorted)))
    logging.info('timesteps: {}'.format(timesteps))
    logging.info('slices: {}'.format(slices))
    lower_boundary = 0
    upper_boundary = slices
    
    spacing_3d = dicoms_sorted[0].sitk_img.GetSpacing()
    # build new spacing with t = 1
    spacing = (spacing_3d[0], spacing_3d[1], spacing_3d[2], 1)
    # create as many 3d volume as timesteps
    for idx in range(timesteps):
        # need to sort every volumelist again, 
        # not all timesteps are the same within one timestep
        image_volume_aslist = dicoms_sorted[lower_boundary:upper_boundary]
        image_volume_aslist = sort_dicoms(image_volume_aslist, sort_for_time=False)

        # extract sitk images from dicom wrapper
        sitk_image_volume_aslist = [img.sitk_img for img in  image_volume_aslist]
        # contours
        contour_volume_aslist = [get_contour(img, contour_as_dict, mixed_mode=False, mask=False) for img in sitk_image_volume_aslist]
        # masks
        mask_volume_aslist = [get_contour(img, contour_as_dict, mixed_mode=False, mask=True) for img in sitk_image_volume_aslist]
        

        # extract numpy array and stack
        # images
        image_nda = np.stack([sitk.GetArrayFromImage(img) for img in sitk_image_volume_aslist], axis=1)
        # contours
        contour_nda = np.stack([sitk.GetArrayFromImage(img) for img in contour_volume_aslist], axis=1)
        # masks
        masks_nda = np.stack([sitk.GetArrayFromImage(img) for img in mask_volume_aslist], axis=1)

        
        # reshape 4d to 3d
        # images
        shape = image_nda.shape
        image_nda_reshaped = np.reshape(image_nda, (shape[1], shape[2], shape[3]))
        # contours
        shape = contour_nda.shape
        contour_nda_reshaped = np.reshape(contour_nda, (shape[1], shape[2], shape[3]))
        # masks
        shape = masks_nda.shape
        mask_nda_reshaped = np.reshape(masks_nda, (shape[1], shape[2], shape[3]))
        
        # append to list and increase boundaries counter
        image_slices_clean.append(sitk.GetImageFromArray(image_nda_reshaped))
        contour_slices.append(sitk.GetImageFromArray(contour_nda_reshaped))
        mask_slices.append(sitk.GetImageFromArray(mask_nda_reshaped))
        
        lower_boundary = lower_boundary + slices
        upper_boundary = upper_boundary + slices

    # join 3dvolumes to 4d volume
    # image
    new_img_clean = sitk.JoinSeries(image_slices_clean)
    new_img_clean.SetSpacing(spacing)
    # contour
    new_contour = sitk.JoinSeries(contour_slices)
    new_contour.SetSpacing(spacing)
    # mask
    new_mask = sitk.JoinSeries(mask_slices)
    new_mask.SetSpacing(spacing)

    size = new_img_clean.GetSize()
    dimension = new_img_clean.GetDimension()
    logging.info('volumes created for {}'.format(args['xml_file']))
    logging.info( "Image size: {}".format(size) )
    logging.info( "Image dimension: {}".format(dimension ))
    logging.info( "Image Spacing: {}".format(spacing ))
    logging.info('Writing images ...')

    sitk.WriteImage( new_img_clean, args['out_file_clean'] )
    logging.info('Image: {} done'.format( args['out_file_clean'] ))
    sitk.WriteImage( new_contour, args['out_file_c'] )
    logging.info('Image: {} done'.format( args['out_file_c'] ))
    sitk.WriteImage( new_mask, args['out_file_m'] )
    logging.info('Image: {} done'.format( args['out_file_m'] ))
    
    logging.info('Patient: {} done!'.format(args['patient']))
    
    
    

In [16]:

def convert_xml_to_nrrd(args):
    
    """
    Entry-point for one extraction, extract contours from one xml file
    and maps them to the corresponding dicom images
    """
    
    path = './reports/'

        # open xml file and create soup
    with open(args['xml_file']) as fp:
        soup = BeautifulSoup(fp, 'lxml')

    # load all segmentation contours
    contour_as_dict = extract_contours(soup)
    logging.info('Converted: {} contours'.format(len(contour_as_dict)))
    
    # load related dicom images
    dicom_images = get_valid_dicom_image(contours=contour_as_dict, args=args)
    dicoms_sorted = sort_dicoms(dicom_images)
    
    # build 4D-Volumes and write them to file system
    build_volume(contour_as_dict, dicoms_sorted, args)

    

In [17]:
def build_config(xml_file, dicom_folder, path_to_export):
    """
    build a dictionary with all directory paths for one Contour-Volume matching
    """
    
    
    f_name_clean = 'volume_clean.nrrd'
    f_name_mixed = 'volume_mixed.nrrd'
    f_name_segment = 'volume_segment.nrrd'
    f_name_mask = 'volume_mask.nrrd'
    dicom_patient = os.path.split(dicom_folder)[1]
    args = {}
    args['patient'] = dicom_patient
    args['xml_file'] = xml_file
    args['src'] = dicom_folder
    args['dest'] = os.path.join(path_to_export, dicom_patient)
    args['out_file_clean'] = os.path.join(path_to_export, dicom_patient + "_" + f_name_clean)
    args['out_mixed'] = os.path.join(path_to_export, dicom_patient + "_" + f_name_mixed)
    args['out_file_c'] = os.path.join(path_to_export, dicom_patient + "_" + f_name_segment)
    args['out_file_m'] = os.path.join(path_to_export, dicom_patient + "_" + f_name_mask)
    
    logging.info('Build config for:')
    logging.info('Xml-file: {}'.format(xml_file))
    logging.info('Dicom-folder: {}'.format(dicom_folder))
    for key,value in args.items():
        logging.info('{}: {}'.format(key, value))
    
    return args

In [18]:
def map_xml_to_study(xml, dicom_study_folders):
    
    """
    extracts patient id etc. from xml file name
    transforms the format and search for that expression in all dicom folders
    """
    # parse patient uid, date from xml file and format it
    xml_file = os.path.basename(xml)
    patient_uid = xml_file[:8]
    logging.debug(patient_uid)
    study_date = xml_file[8:17]
    study_date = study_date[0:4] + "-" + study_date[4:6] + "-" + study_date[6:8]
    logging.debug(study_date)
    
    # match xml file and dicom folder according to the patient-UID and the study date
    matched = [folder for folder in dicom_study_folders if patient_uid in folder and study_date in folder]
    logging.debug(len(matched))
    if matched:
        # if there are two matches, return only the first
        logging.info('Found segmentation for: {}.'.format(matched[0]))
        return xml, matched[0]
    

In [19]:
def create_volumes(path_to_xml, path_to_dicom, path_to_export):
    
    """
    Loads all XML files in path_to_xml
    Lists all patients folders in path_to_dicom_folders
    maps the xml file to patient dicom folder
    start contour extraction and dicom filter, sort and 4d building
    """
    
    xml_extension = '.cvi42wsx'
    
    # get all xml files
    if os.path.isdir(path_to_xml):
        
        xml_files = [os.path.join(path_to_xml, f) for f in os.listdir(path_to_xml) if os.path.isfile(os.path.join(path_to_xml, f))]
        # filter only cvi42wsx files
        xml_files = [file for file in xml_files if os.path.splitext(file)[1] == xml_extension]
        logging.info('Found {} xml-files.'.format(len(xml_files)))
        logging.debug(xml_files)
    else:
        logging.info('Error with path_to_xml: {}'.format(path_to_xml))
        return
    
    # get all study-folders
    if os.path.isdir(path_to_dicom):

        dicom_study_folders = [os.path.join(path_to_dicom, f) for f in os.listdir(path_to_dicom) if os.path.isdir(os.path.join(path_to_dicom, f))]
        logging.info('found {} Patient-Folder.'.format(len(dicom_study_folders)))
        logging.debug(dicom_study_folders)
        
    else:
        logging.info('Error with path_to_dicom: {}'.format(path_to_dicom))
        return
    
    # ensure export path exists
    ensure_dir(path_to_export)
    
    # get a list of (xml-files, study-folder) where we have a segmentation file and dicom images
    segmented_studies = [map_xml_to_study(xml, dicom_study_folders) for xml in xml_files]
    segmented_studies = [folder for folder in segmented_studies if folder is not None]
    logging.info('Matched: {} xml files to a dicom folder'.format(len(segmented_studies)))
    logging.debug(segmented_studies)
    
    # extract contours, filter dicoms, sort slices and build 4d volumes
    for xml_f, folder_n in segmented_studies:
        args = build_config(xml_f, folder_n, path_to_export)
        convert_xml_to_nrrd(args)

    
    

In [20]:
working_dir = os.getcwd()
path_to_xml = os.path.join(working_dir, 'data\\processed\\Segmentations_2\\xml_files\\')
path_to_dicom = 'G:\\sven\\ahf_export\\'
path_to_export = os.path.join(working_dir, 'data\\processed\\nrrd_volume_export_5\\')

create_volumes(path_to_xml, path_to_dicom, path_to_export)




2019-01-18 14:14:50,205 INFO Found 13 xml-files.
2019-01-18 14:14:50,222 INFO found 720 Patient-Folder.
2019-01-18 14:14:50,223 INFO Found segmentation for: G:\sven\ahf_export\0003-04NEJQUZ-2007-03-13.
2019-01-18 14:14:50,224 INFO Found segmentation for: G:\sven\ahf_export\0006-0AE4R74L-1900-01-01.
2019-01-18 14:14:50,224 INFO Found segmentation for: G:\sven\ahf_export\0007-0HQQW4ZN-2007-05-23.
2019-01-18 14:14:50,225 INFO Found segmentation for: G:\sven\ahf_export\0008-0PTV75MP-2005-06-27.
2019-01-18 14:14:50,226 INFO Found segmentation for: G:\sven\ahf_export\0010-0RPELLU8-2007-02-13.
2019-01-18 14:14:50,227 INFO Found segmentation for: G:\sven\ahf_export\0013-13JLP3HN-2007-05-22.
2019-01-18 14:14:50,228 INFO Found segmentation for: G:\sven\ahf_export\0014-1DRHF745-2005-09-01.
2019-01-18 14:14:50,229 INFO Found segmentation for: G:\sven\ahf_export\0016-1E4PF7MR-2006-07-06.
2019-01-18 14:14:50,229 INFO Found segmentation for: G:\sven\ahf_export\0016-1E4PF7MR-2006-07-06.
2019-01-18 14:

2019-01-18 14:15:36,055 INFO Action performed on 105 files
2019-01-18 14:15:36,055 INFO Total Files: 1215
2019-01-18 14:15:36,327 INFO Action performed in sub-directory G:\sven\ahf_export\0003-04NEJQUZ-2007-03-13\1.3.46.670589.11.0.0.11.4.2.0.8365.5.2324.2007031308061384128 done in 0.271s
2019-01-18 14:15:36,328 INFO Action performed on 105 files
2019-01-18 14:15:36,329 INFO Total Files: 1320
2019-01-18 14:15:37,798 INFO Action performed in sub-directory G:\sven\ahf_export\0003-04NEJQUZ-2007-03-13\1.3.46.670589.11.0.0.11.4.2.0.8365.5.2324.2007031308144289242 done in 1.467s
2019-01-18 14:15:37,800 INFO Action performed on 300 files
2019-01-18 14:15:37,801 INFO Total Files: 1620
2019-01-18 14:15:37,853 INFO Action performed in sub-directory G:\sven\ahf_export\0003-04NEJQUZ-2007-03-13\1.3.46.670589.11.0.0.11.4.2.0.8365.5.2324.2007031308144295243 done in 0.050s
2019-01-18 14:15:37,854 INFO Action performed on 9 files
2019-01-18 14:15:37,855 INFO Total Files: 1629
2019-01-18 14:15:37,856 IN

2019-01-18 14:16:15,460 INFO Action performed on 13 files
2019-01-18 14:16:15,460 INFO Total Files: 191
2019-01-18 14:16:15,516 INFO Action performed in sub-directory G:\sven\ahf_export\0006-0AE4R74L-1900-01-01\1.3.12.2.1107.5.99.2.1013.30000008030612131778100068761 done in 0.055s
2019-01-18 14:16:15,517 INFO Action performed on 20 files
2019-01-18 14:16:15,518 INFO Total Files: 211
2019-01-18 14:16:15,575 INFO Action performed in sub-directory G:\sven\ahf_export\0006-0AE4R74L-1900-01-01\1.3.12.2.1107.5.99.2.1013.30000008030612131778100068821 done in 0.056s
2019-01-18 14:16:15,576 INFO Action performed on 20 files
2019-01-18 14:16:15,577 INFO Total Files: 231
2019-01-18 14:16:15,634 INFO Action performed in sub-directory G:\sven\ahf_export\0006-0AE4R74L-1900-01-01\1.3.12.2.1107.5.99.2.1013.30000008030612131778100068881 done in 0.056s
2019-01-18 14:16:15,635 INFO Action performed on 20 files
2019-01-18 14:16:15,636 INFO Total Files: 251
2019-01-18 14:16:15,695 INFO Action performed in s

2019-01-18 14:16:17,337 INFO Total Files: 801
2019-01-18 14:16:17,643 INFO Action performed in sub-directory G:\sven\ahf_export\0006-0AE4R74L-1900-01-01\1.3.12.2.1107.5.99.2.1013.30000008030612131778100070912 done in 0.305s
2019-01-18 14:16:17,644 INFO Action performed on 88 files
2019-01-18 14:16:17,645 INFO Total Files: 889
2019-01-18 14:16:17,959 INFO Action performed in sub-directory G:\sven\ahf_export\0006-0AE4R74L-1900-01-01\1.3.12.2.1107.5.99.2.1013.30000008030612131778100071176 done in 0.312s
2019-01-18 14:16:17,959 INFO Action performed on 88 files
2019-01-18 14:16:17,960 INFO Total Files: 977
2019-01-18 14:16:18,304 INFO Action performed in sub-directory G:\sven\ahf_export\0006-0AE4R74L-1900-01-01\1.3.12.2.1107.5.99.2.1013.30000008030612131778100071440 done in 0.343s
2019-01-18 14:16:18,305 INFO Action performed on 88 files
2019-01-18 14:16:18,305 INFO Total Files: 1065
2019-01-18 14:16:18,619 INFO Action performed in sub-directory G:\sven\ahf_export\0006-0AE4R74L-1900-01-01\

2019-01-18 14:16:20,858 INFO Action performed on 15 files
2019-01-18 14:16:20,859 INFO Total Files: 365
2019-01-18 14:16:20,909 INFO Action performed in sub-directory G:\sven\ahf_export\0006-0AE4R74L-1900-01-01\1.3.12.2.1107.5.99.2.1013.30000008030612131778100069283 done in 0.049s
2019-01-18 14:16:20,910 INFO Action performed on 19 files
2019-01-18 14:16:20,911 INFO Total Files: 384
2019-01-18 14:16:20,969 INFO Action performed in sub-directory G:\sven\ahf_export\0006-0AE4R74L-1900-01-01\1.3.12.2.1107.5.99.2.1013.30000008030612131778100069343 done in 0.057s
2019-01-18 14:16:20,970 INFO Action performed on 20 files
2019-01-18 14:16:20,971 INFO Total Files: 404
2019-01-18 14:16:21,024 INFO Action performed in sub-directory G:\sven\ahf_export\0006-0AE4R74L-1900-01-01\1.3.12.2.1107.5.99.2.1013.30000008030612131778100069403 done in 0.053s
2019-01-18 14:16:21,025 INFO Action performed on 20 files
2019-01-18 14:16:21,026 INFO Total Files: 424
2019-01-18 14:16:21,078 INFO Action performed in s

...

2019-01-18 14:24:20,655 INFO found: 49 contours
2019-01-18 14:24:20,656 INFO read: sarvendocardialContour
2019-01-18 14:24:23,346 INFO found: 47 contours
2019-01-18 14:24:23,347 INFO read: sarvepicardialContour
2019-01-18 14:24:26,044 INFO found: 0 contours
2019-01-18 14:24:26,045 INFO extract all saendocardialContour segmentations
2019-01-18 14:24:26,046 INFO building 49 contours
2019-01-18 14:24:28,296 INFO extract all saepicardialContour segmentations
2019-01-18 14:24:28,297 INFO building 49 contours
2019-01-18 14:24:31,101 INFO extract all sarvendocardialContour segmentations
2019-01-18 14:24:31,103 INFO building 47 contours
2019-01-18 14:24:34,902 INFO Converted: 49 contours
2019-01-18 14:24:34,903 INFO ----------------    Search {} series-ids for dicom directory ----------------
2019-01-18 14:24:36,506 INFO Action performed in sub-directory G:\sven\ahf_export\0018-1E8MCV9L-2005-12-15\1.3.46.670589.11.0.0.11.4.2.0.5167.5.2216.2005121513255714284 done in 1.602s
2019-01-18 14:24:36,

2019-01-18 14:24:51,515 INFO Image: G:\sven\git\cardio\data\processed\nrrd_volume_export_5\0018-1E8MCV9L-2005-12-15_volume_mask.nrrd done
2019-01-18 14:24:51,516 INFO Patient: 0018-1E8MCV9L-2005-12-15 done!
2019-01-18 14:24:51,755 INFO Build config for:
2019-01-18 14:24:51,756 INFO Xml-file: G:\sven\git\cardio\data\processed\Segmentations_2\xml_files\1EMMCVKN200606290931591.3.12.2.1107.5.8.3.485257.836650.48495553.2006062906533535.cvi42wsx
2019-01-18 14:24:51,757 INFO Dicom-folder: G:\sven\ahf_export\0019-1EMMCVKN-2006-06-29
2019-01-18 14:24:51,758 INFO patient: 0019-1EMMCVKN-2006-06-29
2019-01-18 14:24:51,758 INFO xml_file: G:\sven\git\cardio\data\processed\Segmentations_2\xml_files\1EMMCVKN200606290931591.3.12.2.1107.5.8.3.485257.836650.48495553.2006062906533535.cvi42wsx
2019-01-18 14:24:51,759 INFO src: G:\sven\ahf_export\0019-1EMMCVKN-2006-06-29
2019-01-18 14:24:51,760 INFO dest: G:\sven\git\cardio\data\processed\nrrd_volume_export_5\0019-1EMMCVKN-2006-06-29
2019-01-18 14:24:51,761

2019-01-18 14:25:32,821 INFO Total Files: 1951
2019-01-18 14:25:32,926 INFO Action performed in sub-directory G:\sven\ahf_export\0019-1EMMCVKN-2006-06-29\1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062911141201801 done in 0.104s
2019-01-18 14:25:32,927 INFO Action performed on 35 files
2019-01-18 14:25:32,928 INFO Total Files: 1986
2019-01-18 14:25:33,463 INFO Action performed in sub-directory G:\sven\ahf_export\0019-1EMMCVKN-2006-06-29\1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062911154312845 done in 0.534s
2019-01-18 14:25:33,464 INFO Action performed on 140 files
2019-01-18 14:25:33,465 INFO Total Files: 2126
2019-01-18 14:25:33,487 INFO Action performed in sub-directory G:\sven\ahf_export\0019-1EMMCVKN-2006-06-29\1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062911154315846 done in 0.022s
2019-01-18 14:25:33,488 INFO Action performed on 6 files
2019-01-18 14:25:33,489 INFO Total Files: 2132
2019-01-18 14:25:33,644 INFO Action performed in sub-directory G:\sven\ahf_export\0019

2019-01-18 14:25:39,406 INFO Total Files: 1531
2019-01-18 14:25:40,168 INFO Action performed in sub-directory G:\sven\ahf_export\0019-1EMMCVKN-2006-06-29\1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062910550603353 done in 0.761s
2019-01-18 14:25:40,169 INFO Action performed on 150 files
2019-01-18 14:25:40,170 INFO Total Files: 1681
2019-01-18 14:25:40,981 INFO Action performed in sub-directory G:\sven\ahf_export\0019-1EMMCVKN-2006-06-29\1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062911011059513 done in 0.810s
2019-01-18 14:25:40,982 INFO Action performed on 150 files
2019-01-18 14:25:40,983 INFO Total Files: 1831
2019-01-18 14:25:41,381 INFO Action performed in sub-directory G:\sven\ahf_export\0019-1EMMCVKN-2006-06-29\1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062911101070672 done in 0.398s
2019-01-18 14:25:41,382 INFO Action performed on 120 files
2019-01-18 14:25:41,386 INFO Total Files: 1951
2019-01-18 14:25:41,488 INFO Action performed in sub-directory G:\sven\ahf_export\0

2019-01-18 14:26:31,632 INFO Action performed on 256 files
2019-01-18 14:26:31,633 INFO Total Files: 616
2019-01-18 14:26:31,952 INFO Action performed in sub-directory G:\sven\ahf_export\0051-2HQVK81W-2005-03-11\1.3.46.670589.11.0.0.11.4.2.0.5167.5.5892.2005031114072092541 done in 0.318s
2019-01-18 14:26:31,953 INFO Action performed on 90 files
2019-01-18 14:26:31,984 INFO Total Files: 706
2019-01-18 14:26:32,503 INFO Action performed in sub-directory G:\sven\ahf_export\0051-2HQVK81W-2005-03-11\1.3.46.670589.11.0.0.11.4.2.0.5167.5.5892.2005031114111928550 done in 0.517s
2019-01-18 14:26:32,504 INFO Action performed on 105 files
2019-01-18 14:26:32,507 INFO Total Files: 811
2019-01-18 14:26:33,003 INFO Action performed in sub-directory G:\sven\ahf_export\0051-2HQVK81W-2005-03-11\1.3.46.670589.11.0.0.11.4.2.0.5167.5.5892.2005031114144612559 done in 0.495s
2019-01-18 14:26:33,005 INFO Action performed on 105 files
2019-01-18 14:26:33,007 INFO Total Files: 916
2019-01-18 14:26:33,499 INFO 

In [25]:
# save contours and dicom images
# only works if json is returned from DicomFilterSegmentationFile
# save the logs from file walker
with open(os.path.join(working_dir, experiment + '.json'), 'w') as outfile:
    json.dump({experiment : filewalker.open_files}, outfile, indent=2)
logging.info('json dump done, everything fine!')

    # save the contours-object as json

contours_json = {}
for tag, segments in contours.items():
    contours_json[tag] = [vars(contour) for contour in segments]
with open(os.path.join(working_dir, experiment + '_contours.json'), 'w') as fp:
    json.dump({'contours' : contours_json}, fp)

NameError: name 'experiment' is not defined